In [11]:
fr=1000 #frames available in the system
processes = []
import random
import copy
i=1
while(i<10000):
  processes.append([i, random.randint(1,1000), random.randint(1, 20), 0, random.randint(1,10),0]) #0-process, 1-arrival time, 2-pages used, 3-allocated frames, 4-time to complete, 5-time of processing
  i = i+1
processes.sort(key=lambda y:y[1])
print(processes)

[[567, 1, 11, 0, 1, 0], [642, 1, 2, 0, 7, 0], [729, 1, 4, 0, 5, 0], [872, 1, 17, 0, 3, 0], [1191, 1, 1, 0, 1, 0], [1606, 1, 1, 0, 2, 0], [2252, 1, 14, 0, 9, 0], [6229, 1, 8, 0, 5, 0], [7426, 1, 10, 0, 1, 0], [9189, 1, 6, 0, 2, 0], [9344, 1, 17, 0, 7, 0], [3422, 2, 13, 0, 9, 0], [5452, 2, 6, 0, 9, 0], [5933, 2, 16, 0, 3, 0], [7247, 2, 20, 0, 7, 0], [8541, 2, 10, 0, 7, 0], [9116, 2, 6, 0, 3, 0], [311, 3, 19, 0, 5, 0], [537, 3, 1, 0, 5, 0], [810, 3, 4, 0, 8, 0], [2088, 3, 2, 0, 3, 0], [6317, 3, 20, 0, 7, 0], [6803, 3, 19, 0, 1, 0], [8062, 3, 7, 0, 10, 0], [9534, 3, 10, 0, 2, 0], [9690, 3, 12, 0, 5, 0], [2287, 4, 12, 0, 2, 0], [4113, 4, 16, 0, 2, 0], [6795, 4, 19, 0, 7, 0], [7279, 4, 4, 0, 7, 0], [8329, 4, 1, 0, 8, 0], [8948, 4, 4, 0, 1, 0], [9659, 4, 2, 0, 4, 0], [64, 5, 5, 0, 5, 0], [764, 5, 5, 0, 8, 0], [2970, 5, 1, 0, 9, 0], [3542, 5, 2, 0, 8, 0], [4855, 5, 1, 0, 9, 0], [6627, 5, 19, 0, 5, 0], [6676, 5, 7, 0, 3, 0], [6986, 5, 1, 0, 10, 0], [8504, 5, 12, 0, 1, 0], [426, 6, 8, 0, 10, 0],

Random - the number of frames allocated for each proccess is chosen at random. My implementation works under the assumption that every page accessed by a process will be unique.

In [12]:
import random
import copy
processes.sort(key=lambda y:y[1]) #sort by arrival time
rand = copy.deepcopy(processes)
donepr = [] #this will keep track of running processes
totalsize = 0 #this is the total amount of frames allocated to running processes
rpf = 0
index=0
at=0
time=0
for i in rand:
    if(i[1]>time):             #time is overwriten whenever a process with higher arrival time is the 'i' one
      for j in donepr:
        j[5]=j[5]+(i[1]-time)  #time of processing increases
        if(j[5]>=j[4]):        #if the process is finished it's removed from the array and its frames can be realocated
          totalsize=totalsize-j[3]
          donepr.remove(j)
      time=i[1]
    alloc = random.randint(1, fr) #the number of allocated frames is chosen at random
    totalsize += alloc
    if totalsize > fr:            #if there are not enough frames in the system to allocate...
        rpf += i[2]
        index=max(i[0], index)
        at=max(at, i[1])
        while totalsize>fr:       #...time starts running
          for j in donepr:
            j[5]=j[5]+1
            if(j[5]==j[4]):
              totalsize=totalsize-j[3]
              donepr.remove(j)
            at+=1
            time+=1
        if(index<len(rand)):
          while rand[index][1]<=at and index<len(rand): #this while adds pagefaults for all the processes which were not able to get enough frames at
            at=rand[index][1]
            rpf+=rand[index][2]
            index+=1
            if(index>=len(rand)):
              break
    i[3] = alloc
    donepr.append(i)
    print(f"Allocated {alloc} frames to process {i[0]}. Total size: {totalsize}")
print(donepr)
print(f"Generated page faults = {rpf}")

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Allocated 338 frames to process 5185. Total size: 977
Allocated 468 frames to process 5924. Total size: 468
Allocated 897 frames to process 6480. Total size: 897
Allocated 908 frames to process 6677. Total size: 908
Allocated 334 frames to process 6869. Total size: 334
Allocated 513 frames to process 8340. Total size: 847
Allocated 309 frames to process 8519. Total size: 822
Allocated 224 frames to process 1409. Total size: 737
Allocated 686 frames to process 1558. Total size: 910
Allocated 962 frames to process 1742. Total size: 962
Allocated 647 frames to process 2899. Total size: 647
Allocated 275 frames to process 3947. Total size: 922
Allocated 641 frames to process 4011. Total size: 641
Allocated 58 frames to process 4398. Total size: 699
Allocated 214 frames to process 6128. Total size: 913
Allocated 617 frames to process 6548. Total size: 831
Allocated 547 frames to process 6845. Total size: 547
Allocated 284 fram

Proportional - the frame allocation depends on the size of the processes. The number of allocated frames for each process is calculated by dividing its size by the sum of all sizes, the result of which is then multiplied by the total amount of frames. My implementation  again works under the assumption that every page accessed by a process will be unique.

In [13]:
import random
import copy
ppf=0
totalsize=0
donepr=[]
processes.sort(key=lambda y:y[1]) #sort by arrival time
prop=copy.deepcopy(processes)
time=0
at=0
index=0
totalsizep=0  #size of the process in terms of pages
print(prop)
for i in prop:
  if(i[1]>time):
      for j in donepr:
        j[5]=j[5]+(i[1]-time)
        if(j[5]>=j[4]):
          totalsize=totalsize-j[3]
          totalsizep=totalsizep-j[2]
          donepr.remove(j)
      time=i[1]
  totalsizep=totalsizep+i[2]
  if(len(donepr)==0 or totalsizep==0): #if there are no processes running, the amount of allocated frames is equal to the amount of pages of the process
    alloc=i[2]
  else:
    alloc=int((i[2]/totalsizep)*fr)  #otherwise it's calculated proportionally
  totalsize+=alloc
  if(alloc<i[2]):   #if not all pages have access to a frame, paging errors occur
      totalsize=totalsize+(i[2]-alloc)
      ppf+=(i[2]-alloc)
  if(totalsize>fr):
    ppf=ppf+i[2]
    index=max(i[0], index)
    at=max(at, i[1])
    while totalsize>fr: #this is the same as in the previous one
      for j in donepr:
        j[5]=j[5]+1
        if(j[5]==j[4]):
          totalsize=totalsize-j[3]
          totalsizep=totalsizep-j[2]
          donepr.remove(j)
        at+=1
        time+=1
    if(index<len(prop)):
      while prop[index][1]<=at and index<len(prop):
        at=prop[index][1]
        ppf+=prop[index][2]
        index+=1
        if(index>=len(prop)):
          break
  i[3] = alloc
  donepr.append(i)
  print(f"Allocated {alloc} frames to process {i[0]}. Total size: {totalsize}")
print(donepr)
print(f"Generated page faults = {ppf}")

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Allocated 66 frames to process 5185. Total size: 956
Allocated 116 frames to process 5924. Total size: 905
Allocated 115 frames to process 6480. Total size: 844
Allocated 140 frames to process 6677. Total size: 984
Allocated 65 frames to process 6869. Total size: 983
Allocated 7 frames to process 8340. Total size: 990
Allocated 97 frames to process 8519. Total size: 886
Allocated 32 frames to process 1409. Total size: 918
Allocated 75 frames to process 1558. Total size: 993
Allocated 22 frames to process 1742. Total size: 922
Allocated 81 frames to process 2899. Total size: 678
Allocated 42 frames to process 3947. Total size: 720
Allocated 144 frames to process 4011. Total size: 864
Allocated 8 frames to process 4398. Total size: 872
Allocated 145 frames to process 6128. Total size: 763
Allocated 85 frames to process 6548. Total size: 848
Allocated 45 frames to process 6845. Total size: 893
Allocated 59 frames to process 

Page fault frequency - in this algorithm, the frequency is calculated by dividing the amount of paging errors happening for each process by some chosen period of time. When the frequency exceeds a certain threshold, another frame is allocated towards that process, and when it goes below a thresholds a frame is released. If there are no more frames available the process is halted.

In [17]:
import random
import copy
pfpf=0
totalsize=0
donepr=[]
processes.sort(key=lambda y:y[1]) #sort by arrival time
pff=copy.deepcopy(processes)
time=0
at=0
index=0
low=1  #the low threshold
high=2 #the high threshold
stop=8 #the stopping threshold
totalsizep=0
print(pff)
for i in pff:
  if(i[1]>time):
      for j in donepr:
        j[5]=j[5]+(i[1]-time)
        if(j[5]>=j[4]):
          totalsize=totalsize-j[3]
          totalsizep=totalsizep-j[2]
          donepr.remove(j)
      time=i[1]
  if(len(donepr)==0):                     #
    alloc=i[2]                            #
  else:                                   # proportional allocation
    alloc=int((i[2]/totalsizep)*fr)       #
  totalsize+=alloc                        #
  totalsizep+=i[2]
  if(totalsize>fr):
    pfpf+=(totalsize-fr)
    freq=(totalsize-fr)/i[4] #calculating the frequency
  else:
    freq=0  #if there are no page faults frequency=0
  if(freq<low):
    totalsize-=alloc  #since I wasn't really able to make the algorithm work with increasing and ecreasing frames by 1, it instead "predicts" the amount of frames it will reach (so pretty much just allocates the needed amount right away)
    alloc=i[2]
    totalsize+=alloc
  if(freq>high and freq<stop):
    totalsize-=alloc
    alloc=i[2]
    totalsize+=alloc
  if(freq>=stop):
    pfpf+=i[2]    #if it'll be stopped it's not taken into account, it just counts towards page faults
  else:
    donepr.append(i)
  while totalsize>fr:
    for j in donepr:
      j[5]=j[5]+1
      if(j[5]==j[4]):
        totalsize=totalsize-j[3]
        totalsizep=totalsizep-j[2]
        donepr.remove(j)
      at+=1
      time+=1
  if(index<len(pff)):
    while pff[index][1]<=at and index<len(pff):
      at=pff[index][1]
      pfpf+=pff[index][2]
      index+=1
      if(index>=len(pff)):
        break
  i[3]=alloc
  print(f"Allocated {alloc} frames to process {i[0]}. Total size: {totalsize}")
print(donepr)
print(f"Generated page faults = {pfpf}")

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Allocated 8 frames to process 5185. Total size: 695
Allocated 16 frames to process 5924. Total size: 711
Allocated 15 frames to process 6480. Total size: 726
Allocated 18 frames to process 6677. Total size: 744
Allocated 9 frames to process 6869. Total size: 753
Allocated 1 frames to process 8340. Total size: 754
Allocated 14 frames to process 8519. Total size: 768
Allocated 4 frames to process 1409. Total size: 666
Allocated 10 frames to process 1558. Total size: 676
Allocated 3 frames to process 1742. Total size: 679
Allocated 11 frames to process 2899. Total size: 690
Allocated 4 frames to process 3947. Total size: 694
Allocated 16 frames to process 4011. Total size: 710
Allocated 1 frames to process 4398. Total size: 711
Allocated 19 frames to process 6128. Total size: 730
Allocated 9 frames to process 6548. Total size: 739
Allocated 5 frames to process 6845. Total size: 744
Allocated 7 frames to process 7997. Total s

Working set - for this algorithm during a certain period of time the paging sequence is checked for duplicate pages. The amount of original pages which were excessed will be the amount of frames allocated to the process. When there are no more frames available, a process (in my case chosen in the fifo fashion) is stopped and the frames it used are alocated in a proportional fashion to other processes that are currently working. My implementation is a bit simpler though, and instead of distributing the frames among the running processes, it just frees them up for the upcoming ones and the process itself is set to attempt to continue in the next second of the simulation.

In [20]:
import random
import copy
wpf=0
totalsize=0
donepr=[]
processes.sort(key=lambda y:y[1]) #sort by arrival time
ws=copy.deepcopy(processes)
time=0
at=0
index=0
print(ws)
for i in ws:
  if(i[1]>time):
      for j in donepr:
        j[5]=j[5]+(i[1]-time)
        if(j[5]>=j[4]):
          totalsize=totalsize-j[3]
          donepr.remove(j)
      time=i[1]
  pages=[]
  while len(pages)!=i[2]:
    pages.append(random.randint(1,100)) #we generate a set of pages
  counter=0
  diff=0
  alldiff=0
  temp=[]
  delta=int((i[2]/i[4])*(0.75*i[4])) #the working set is based on the 3/4 of the time it takes to finish each process
  for j in pages:
   if(not (j in temp)):
    counter+=1
    temp.append(j)
    alldiff+=1   #now we're checking how many of them are unique
    if(counter<delta):
      diff+=1    #and how many in the 0.75 of the time are unique
  alloc=diff
  totalsize+=alloc
  i[3]+=alloc
  if((alldiff-diff)>=0): #if not all pages have access to frames page faults are added
    wpf+=(alldiff-diff)
    totalsize+=(alldiff-diff)
    if(totalsize<fr):
      i[3]+=alldiff-diff
      alloc+=(alldiff-diff)
    else:
      while(totalsize>fr): #when there aren't enough frames the processes are halted and moved in a fifo fashion (so starting with the one at the beginning of donepr)
        temp2=donepr[0][3]
        donepr[0][1]=time+1
        temp3 = ws[ws.index(i)+1]
        while ws[temp3][1]!=time+1:
          temp3+=1
        ws.insert(temp3,donepr[0])
        donepr.pop(0)
  donepr.append(i)
  print(f"Allocated {alloc} frames to process {i[0]}. Total size: {totalsize}")
print(donepr)
print(f"Generated page faults = {wpf}")

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Allocated 7 frames to process 5185. Total size: 656
Allocated 15 frames to process 5924. Total size: 671
Allocated 15 frames to process 6480. Total size: 686
Allocated 15 frames to process 6677. Total size: 701
Allocated 8 frames to process 6869. Total size: 709
Allocated 1 frames to process 8340. Total size: 710
Allocated 11 frames to process 8519. Total size: 721
Allocated 4 frames to process 1409. Total size: 623
Allocated 10 frames to process 1558. Total size: 633
Allocated 3 frames to process 1742. Total size: 636
Allocated 10 frames to process 2899. Total size: 646
Allocated 4 frames to process 3947. Total size: 650
Allocated 14 frames to process 4011. Total size: 664
Allocated 1 frames to process 4398. Total size: 665
Allocated 19 frames to process 6128. Total size: 684
Allocated 9 frames to process 6548. Total size: 693
Allocated 5 frames to process 6845. Total size: 698
Allocated 6 frames to process 7997. Total s

In terms of page faults from the highest to lowest the algorithms rank: random, proportional, working set (for the time period of 0.5+ of the time it takes for the process to finish - as it decreases there are more page faults). I'm not including page fault frequency, as the amount of page faults for that one is always close to 0, which indicates that either my simplifications were too much, or I just messed something up. The other results seem reasonable though.